First we'll import a few necessary modules.

In [7]:
import fbchat
import os

Next I'll define a simple function to return all messages between myself and a friend. If you'd like to do this, run the following code to set up an instance of the fbchat client with your fb ID and password exposed as environment variables. 
```python
client = fbchat.Client(os.environ['ID'], os.environ['PASSWORD'])
friend_name = input("Friend's name: ")

f = pull_messages(client, friend_name)
```

In [8]:
def pull_messages(client, friend_name):
    friendList = client.getUsers(friend_name)
    friend = friendList[0]

    last_messages = client.getThreadInfo(friend.uid, 10000) # 10000 should cover all messages?
    last_messages.reverse()  # messages come in reversed order

    result = []

    for message in last_messages:
        if message.author.split(":")[1] == str(os.environ['ID']):
            name = "You"
        else: 
            name = friend_name
        if hasattr(message, 'body') and message.body:
            result.append(name + ": " + message.body)

    return result

In [9]:
f = open('messages.txt', 'r')

We'll use the package vaderSentiment (https://pypi.python.org/pypi/vaderSentiment) to characterize messages between the user and a friend. The package has been trained using tweets and classifies text as positive, negative, or neutral. We then define a couple of dictionaries to keep track of our data.

In [10]:
from vaderSentiment import vaderSentiment
ana = vaderSentiment.SentimentIntensityAnalyzer()

you = {
    'total':0,
    'pos':0,
    'neg':0,
    'neut':0}

friend = {
    'total':0,
    'pos':0,
    'neg':0,
    'neut':0}

Here's where most of the work is done. We go through every message and classify it as positive, negative, or neutral for both the user and the user's friend.

In [11]:
for msg in f:
    split = msg.split(':')

    if len(split) > 1:

        classified = ana.polarity_scores(split[1])

        if split[0] == 'You':
            you['total'] += 1
            if classified['neg'] > 0.5:
                you['neut'] += 1
            elif classified['pos'] > 0.5:
                you['pos'] += 1
            else: # classified['neg'] >= 0.5
                you['neg'] += 1
        else:
            friend['total'] += 1
            if classified['neg'] > 0.5:
                friend['neut'] += 1
            elif classified['pos'] > 0.5:
                friend['pos'] += 1
            else: # classified['neg'] >= 0.5
                friend['neg'] += 1

At this point, there's already a ton we can do with the data we have! Let's go a few steps further though.

